In [10]:
"""环境验证"""
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
import torch

print("环境配置成功！")

环境配置成功！


In [12]:
"""实现简单的RAG模型"""
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM

# 加载本地模型
embed_model_path = "/home/work/var/data/ssr-share-data/m3e-base/"
llm_model_path = "/home/work/var/data/ssr-share-data/Qwen1.5-7B-Chat/"

# 加载文档
documents = SimpleDirectoryReader('temp_docs').load_data()

# 初始化嵌入模型
embed_model = HuggingFaceEmbedding(model_name=embed_model_path)


# 初始化大语言模型
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt

def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"


llm = HuggingFaceLLM(
    model_name=llm_model_path,
    tokenizer_name=llm_model_path,
    context_window=3900,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95, "pad_token_id": 151645},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)
# 测试大模型的输出
response = llm.complete("What is the meaning of life?")
print(str(response))

# 创建索引
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# 创建查询引擎
query_engine = index.as_query_engine(llm=llm)
print(f"query_engine: {query_engine}")

# 测试RAG系统
test_query = "法国有什么著名的建筑？"
response = query_engine.query(test_query)
print(f"问题: {test_query}")
print(f"回答: {response}")

No sentence-transformers model found with name /home/work/var/data/ssr-share-data/m3e-base/. Creating a new one with MEAN pooling.
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.69s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.




As an AI language model, I don't have a definitive answer to this question as it's a philosophical and existential one that has been debated throughout history. Different individuals, cultures, religions, and belief systems offer their own interpretations.

Some may find meaning in personal fulfillment through achieving goals, helping others, contributing to society, or pursuing their passions. Others may find it in spirituality, connection to a higher power, or following a moral code. For some, life may be simply about experiencing and enjoying the world around them.

Ultimately, the meaning of life is a subjective concept that each person must determine for themselves based on their own values, beliefs, and life experiences. It's a lifelong journey that may evolve and change over time.
query_engine: <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x7f71c1151b70>
问题: 法国有什么著名的建筑？
回答: 埃菲尔铁塔是法国的著名建筑，位于巴黎，是世界上最知名的建筑物之一，也是巴黎的标志。另外，自由女神像是法国赠送给美国的，但这座雕像

In [13]:
"""评估基础RAG系统性能"""
test_questions = [
    "中国有什么著名的建筑？",
    "印度有什么知名的地标？",
    "美国的标志性建筑是什么？",
    "哪个地标建于15世纪？"
]

for question in test_questions:
    response = query_engine.query(question)
    print(f"问题: {question}")
    print(f"回答: {response}")
    print("-" * 50)

问题: 中国有什么著名的建筑？
回答: 中国的著名建筑有长城和埃菲尔铁塔。长城是中国古代伟大的防御工程，世界上最长的城墙，被誉为世界七大奇迹之一；而埃菲尔铁塔位于法国巴黎，是巴黎的标志性建筑，也是世界上最知名的建筑物之一。
--------------------------------------------------
问题: 印度有什么知名的地标？
回答: 印度的知名地标包括泰姬陵，它位于阿格拉，是莫卧儿帝国皇帝沙贾汗为纪念他的妻子姆塔兹·玛哈尔建造的白色大理石陵墓。另外还有自由女神像，虽然这个地标在美国，但因其与美国的关系，也是印度游客常提及的国际知名建筑。
--------------------------------------------------
问题: 美国的标志性建筑是什么？
回答: 美国的标志性建筑有两座，分别是自由女神像和埃菲尔铁塔。自由女神像是美国的国家标志之一，位于纽约港口的自由岛上；而埃菲尔铁塔虽然位于法国，但因其知名度享誉全球，被视为美国的文化代表。
--------------------------------------------------
问题: 哪个地标建于15世纪？
回答: 马丘比丘
--------------------------------------------------
